# Loading the Data and Check Structure

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from convertdate import persian
from datetime import datetime
import jdatetime

In [2]:
file_path = 'P2_user_segmentation.xlsx'
data = pd.read_excel(file_path)

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 106 entries, Mobile Key to AllGuild
dtypes: datetime64[ns](2), float64(77), int64(3), object(24)
memory usage: 40.4+ MB
None


In [4]:
print(data.columns)

Index(['Mobile Key', 'Person Is Alive', 'Person Birth Date',
       'Person Gender Title', 'Person First Name', 'Person Last Name',
       'Mobile Number Encrypt', 'Mobile Number Masked', 'FirstAppInstallDate',
       'Mobile Operator Id',
       ...
       'CMS_Balance', 'LastTrsDateWallet', 'FirstProvince', 'SecondProvince',
       'FirstRegion', 'SecondRegion', 'FirstGuild', 'SecondGuild',
       'ThirdGuild', 'AllGuild'],
      dtype='object', length=106)


# Finding duplicate columns

In [5]:
duplicate_columns = data.columns[data.columns.duplicated()].unique()

if len(duplicate_columns) > 0:
    print("Duplicate column names found:")
    for col in duplicate_columns:
        print(col)
else:
    print("No duplicate column names found")

No duplicate column names found


# Replacing outliers with NaN

In [6]:
def replace_outliers_with_nan(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df[col] = df[col].apply(lambda x: np.nan if (x < lower_bound) or (x > upper_bound) else x)
    
    return df

data = replace_outliers_with_nan(data)
print(data.head())

   Mobile Key Person Is Alive Person Birth Date Person Gender Title  \
0           2            True        1997-10-09                 مرد   
1           4             NaN               NaT                 NaN   
2           6             NaN               NaT                 NaN   
3           7            True        1979-07-13                 مرد   
4           9            True        1979-03-11                 مرد   

  Person First Name Person Last Name     Mobile Number Encrypt  \
0              مجید             ملکی  HN8x8N88m+sikOUIs4Iuxg==   
1               NaN              NaN  /Y38H9ETC2tMantBTYBQzw==   
2               NaN              NaN  ThuZBrT+/BueLw7LxSX15w==   
3           حمیدرضا     عزیزی مربویه  0sEbFPeIKs9cCJLA73cOoQ==   
4           امیدعلی          احمدپور  FZT2oVrc5UCFaN8gjELmCg==   

  Mobile Number Masked     FirstAppInstallDate  Mobile Operator Id  ...  \
0          0938***0926 2017-04-18 20:15:22.910                   2  ...   
1          0913***9963 201

# Analyzing Missing Values

**Calculating the percentage of missing values for each column**

In [7]:
missing_values = data.isnull().mean() * 100
print(missing_values)

Mobile Key              0.000
Person Is Alive        67.638
Person Birth Date      68.842
Person Gender Title    67.702
Person First Name      67.638
                        ...  
SecondRegion           79.914
FirstGuild             76.706
SecondGuild            78.704
ThirdGuild             82.046
AllGuild               76.706
Length: 106, dtype: float64


**Identifying columns with high missing rates (above 70%)**

In [8]:
high_missing_columns = missing_values[missing_values > 70].sort_values(ascending=False)
print(high_missing_columns)

FirstTrsDateIntFlight    99.968
CountIntFlight           99.966
AmountIntFlight          99.966
LastTrsDateIntFlight     99.962
FirstTrsDateDFlight      99.526
                          ...  
CountC2C                 72.992
TotalAmountPos           71.972
FirstTrsDatePos          71.558
LastTrsDateC2C           71.256
TotalTrsPos              71.192
Length: 73, dtype: float64


# Handling Missing Values

**Droping columns with more than 70% missing values**

In [9]:
data = data.drop(columns=high_missing_columns[high_missing_columns > 70].index)

**Droping rows with more than 50% missing values**

In [10]:
threshold = data.shape[1] / 2
data = data.dropna(thresh=threshold)

# Imputation Methods

**Datetime Columns (Forward Fill)**

In [11]:
datetime_columns = data.select_dtypes(include=['datetime64[ns]']).columns
data[datetime_columns] = data[datetime_columns].fillna(method='ffill')

C:\Users\SURFACE\AppData\Local\Temp\ipykernel_7556\1921836317.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[datetime_columns] = data[datetime_columns].fillna(method='ffill')


**Numerical Columns (K-Nearest Neighbors)**

In [12]:
imputer = KNNImputer(n_neighbors=5)
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
data[numerical_columns] = imputer.fit_transform(data[numerical_columns])

**Categorical Columns (mode imputation)**

In [13]:
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
for col in categorical_columns:
    data[col] = data[col].fillna(data[col].mode()[0])

C:\Users\SURFACE\AppData\Local\Temp\ipykernel_7556\4199709154.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].fillna(data[col].mode()[0])


# Data Type Correction

In [14]:
print(data[['FirstTrsDate', 'LastTrsDate']].dtypes)

FirstTrsDate    float64
LastTrsDate     float64
dtype: object


In [15]:
data['FirstTrsDate'] = np.floor(data['FirstTrsDate']).astype(int)
data['LastTrsDate'] = np.floor(data['LastTrsDate']).astype(int)
data[['FirstTrsDate', 'LastTrsDate']].head()

,FirstTrsDate,LastTrsDate
0,14010107,14020326
3,14010101,14020501
4,14010101,14020501
5,14010243,14020108
8,14011128,14020214


In [16]:
def fix_date(date_str):
    date_str = str(date_str)

    year = int(date_str[:4])
    month = int(date_str[4:6])
    day = int(date_str[6:])

    if day < 1:
        day = 1
    
    if day > 31:
        day = 29

    if month < 1:
        month = 1
    
    if month > 12:
        month = 12
    
    if month == 12 and day == 30:
        day = 29

    

    return f"{year:04d}{month:02d}{day:02d}"

data['FirstTrsDate'] = data['FirstTrsDate'].apply(fix_date)
data['LastTrsDate'] = data['LastTrsDate'].apply(fix_date)
data[['FirstTrsDate', 'LastTrsDate']].head()

,FirstTrsDate,LastTrsDate
0,14010107,14020326
3,14010101,14020501
4,14010101,14020501
5,14010229,14020108
8,14011128,14020214


In [19]:
def persian_to_english(persian_date_str):
    try:
        persian_year = int(persian_date_str[:4])
        persian_month = int(persian_date_str[4:6])
        persian_day = int(persian_date_str[6:])

        persian_date = jdatetime.date(persian_year, persian_month, persian_day)
        english_date = persian_date.togregorian()
        
        return english_date
    except ValueError as e:
        if 'day is out of range for month' in str(e):
            print(f"Error with Persian date: {persian_date_str}")
        raise  


try:
    data['FirstTrsDate'] = data['FirstTrsDate'].apply(persian_to_english)
    data['LastTrsDate'] = data['LastTrsDate'].apply(persian_to_english)
except ValueError:
    print("There was an error in the conversion.")

Error with Persian date: 14010731
There was an error in the conversion.


# Saving Excel File

In [20]:
data.to_excel("Preprocessed Data.xlsx", index=False)